In [ ]:
%pylab inline

# Reweighting

**hep_ml.reweight** contains methods to reweight distributions: `BinsReweighter`, `GBReweighter`

**hep_ml.metrics_utils** contains `ks_2samp_weighted`. Use it to compute KS metric

In [ ]:
from hep_ml import reweight
import root_numpy
import pandas
from hep_ml.metrics_utils import ks_2samp_weighted

## Downloading data

In [ ]:
storage = 'https://github.com/arogozhnikov/hep_ml/blob/data/data_to_download/'
!wget -O datasets/MC_distribution.root -nc $storage/MC_distribution.root?raw=true
!wget -O datasets/RD_distribution.root -nc $storage/RD_distribution.root?raw=true

## Read data

Pay attention that here we work with `.root` files and `root_numpy` can help

In [ ]:
columns = ['hSPD', 'pt_b', 'pt_phi', 'vchi2_b']

original = root_numpy.root2array('datasets/MC_distribution.root', branches=columns)
target = root_numpy.root2array('datasets/RD_distribution.root', branches=columns)

original = pandas.DataFrame(original)
target = pandas.DataFrame(target)

original_weights = numpy.ones(len(original))

## Original distributions

In [ ]:
hist_kw = {'bins': 100, 'normed': True, 'alpha': 0.5}

def plot_pdf(features, new_original_weights, weights_target=None, label=''):
    figsize(14, 8)
    if weights_target is None:
        weights_target = numpy.ones(len(target), dtype=float)
    for index, column in enumerate(features, 1):
        xlim = numpy.percentile(target[column], [0.01, 99.99])
        subplot(2, 2, index)
        hist(original[column].values, weights=new_original_weights, range=xlim, 
             label=label + 'original(MC)', **hist_kw)
        hist(target[column].values, range=xlim, label='target(real)', **hist_kw)
        title(column)
        legend()
        print column, 'KS:', ks_2samp_weighted(original[column], target[column], 
                                               weights1=new_original_weights, 
                                               weights2=weights_target)        

In [ ]:
plot_pdf(columns, original_weights)

## Compare 1D reweighting: Bin, GB. 

* Choose one variable for reweighting
* Use KS metric to compare which method is the best.
* Do other variables after 1D reweigthig agree? What does it mean?

### Bin reweighting

In [ ]:
variable_1d = ...
# play with it
bins_reweighter = reweight.BinsReweighter(...)
bins_reweighter.fit(original[variable_1d], target[variable_1d])

bins_weights = bins_reweighter.predict_weights(original[variable_1d])
plot_pdf(columns, bins_weights, label='Bin 1D: ')

As you see for some of other variables KS metric became a bit worse, for some - a bit better. The Difference is inessential.

**TODO**

* Compute CvM between chosen feature and others. Are they correlated?

In [ ]:
from utils import compute_cvm
...

For a random variable the correlation is 10 times less than for features thus all features are correlated indeed.

### GB reweighting

This algorithm is inspired by gradient boosting and is able to fight curse of dimensionality.
It uses decision trees and special loss functiion (**ReweightLossFunction**).

**GBReweighter** supports negative weights (to reweight MC to splotted real data).

In [ ]:
reweighter = reweight.GBReweighter(...)
reweighter.fit(original[variable_1d], target[variable_1d])

gb_weights = reweighter.predict_weights(original[variable_1d])
plot_pdf(columns, gb_weights, label='GB 1D: ')

## Compare ND reweighting: Bin, GB. 

Use ML to compare which method is the best. Does it really work?

### Bin ND reweighter

In [ ]:
...

### GB ND reweighter

In [ ]:
...

## GB-discrimination
let's check how well the classifier is able to distinguish these distributions. ROC AUC is taken as measure of quality.

For this puprose we split data into train and test, then train a classifier to distinguish these distributions.
If ROC AUC = 0.5 on test, distibutions are equal, if ROC AUC = 1.0, they are ideally separable.

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_auc_score

data = numpy.concatenate([original, target])
labels = numpy.array([0] * len(original) + [1] * len(target))

weights = {}
weights['original'] = original_weights
weights['bins'] = bins_weights
weights['gb_weights'] = gb_weights


for name, new_weights in weights.items():
    W = numpy.concatenate([new_weights / new_weights.sum() * len(target), [1] * len(target)])
    Xtr, Xts, Ytr, Yts, Wtr, Wts = train_test_split(data, labels, W, random_state=42, train_size=0.51)
    clf = GradientBoostingClassifier(...).fit(Xtr, Ytr, sample_weight=Wtr)
    
    print name, roc_auc_score(Yts, clf.predict_proba(Xts)[:, 1], sample_weight=Wts)

Thus GB reweighter is better because its AUC is close to 0.5. And it strongly improves original pdf while Bin reweighter improves a little bit.

## Comparing some simple expressions:
The most interesting is checking some other variables in multidimensional distributions (those are expressed via original variables).

* Compute different expressions. For them compute KS.
* Make the `hist2d` plots for one combination of features vs another combination of features

**Hint**:
`hist2d(original.eval(expression1).values, original.eval(expression2).values, 
        bins=100, normed=True)`

In [ ]:
def check_ks_of_expression(expression):
    col_original = original.eval(expression, engine='python')
    col_target = target.eval(expression, engine='python')
    w_target = numpy.ones(len(col_target), dtype='float')
    print 'No reweight   KS:', ks_2samp_weighted(col_original, col_target, weights1=original_weights, weights2=w_target)        
    print 'Bins reweight KS:', ks_2samp_weighted(col_original, col_target, weights1=bins_weights, weights2=w_target)
    print 'GB Reweight   KS:', ks_2samp_weighted(col_original, col_target, weights1=gb_weights, weights2=w_target)

### Check KS for simple expressions

In [ ]:
check_ks_of_expression('hSPD')

In [ ]:
check_ks_of_expression('hSPD * pt_phi')

In [ ]:
# yours expressions

### Hist2d plots 

In [ ]:
def hist2d_expressions(f1, f2, vmax):
    figsize(15, 12)
    
    subplot(2, 2, 1)
    hist2d(target.eval(f1).values, target.eval(f2).values, 
           bins=100, normed=True)
    clim(0, vmax)
    title('target: {} vs {}'.format(f1, f2), color='y')
    
    subplot(2, 2, 2)
    hist2d(original.eval(f1).values, original.eval(f2).values, 
           bins=100, normed=True)
    clim(0, vmax)
    title('original: {} vs {}'.format(f1, f2), color='y')
    xlim(min(target.eval(f1).values), max(target.eval(f1).values))
    ylim(min(target.eval(f2).values), max(target.eval(f2).values))
    
    subplot(2, 2, 4)
    hist2d(original.eval(f1).values, original.eval(f2).values, 
           bins=100, normed=True, 
           weights=bins_weights)
    clim(0, vmax)
    title('Bin RW: {} vs {}'.format(f1, f2), color='y')
    xlim(min(target.eval(f1).values), max(target.eval(f1).values))
    ylim(min(target.eval(f2).values), max(target.eval(f2).values))
    
    subplot(2, 2, 3)
    hist2d(original.eval(f1).values, original.eval(f2).values, 
           bins=100, normed=True, 
           weights=gb_weights)
    clim(0, vmax)
    xlim(min(target.eval(f1).values), max(target.eval(f1).values))
    ylim(min(target.eval(f2).values), max(target.eval(f2).values))
    title('GB RW: {} vs {}'.format(f1, f2), color='y')

In [ ]:
hist2d_expressions('hSPD * pt_phi', 'vchi2_b', 1e-5)

In [ ]:
hist2d_expressions('(pt_b * pt_phi) ** (0.1)', 'hSPD', 0.005)

In [ ]:
hist2d_expressions('pt_b **0.5', 'pt_phi ** 0.1', 3)

In [ ]:
hist2d_expressions(...)

#### What reweighter is better for feature combinations?

As you can see, GB reweighter is better by its KS for expressions, also for hist2d it looks better and is more similar to the target.

# Build classifier, which agree on MC and real

It is time to apply our knowledge to analyze $\tau\to3\mu$ without only removing disagreement features!

**TODO: **

* analyze $\tau\to3\mu$ data to build classifier which pass agreement threshold 0.09
* compare models trained on disagreement features, without them, with reweigthing (Bin, GB)
* compute AUC and don't forget use weights during the AUC computation if you reweighted data.
* Does reweighting only by one feature `SPDhits` help to pass agreement threshold?
* Compare MC signal vs MC control to avoid systematic error

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

In [ ]:
data = pandas.read_csv('datasets/training.csv')
data_agreement = pandas.read_csv('datasets/check_agreement.csv')
train_features = list(set(data_agreement.columns) - {'id', 'signal', 'weight'})

### Compute KS for all features to find the most disagreement

In [ ]:
weights_mc_control = data_agreement[data_agreement.signal == 1]['weight']
weights_rd_control = data_agreement[data_agreement.signal == 0]['weight']

In [ ]:
def compute_ks_for_all(features, weights):
    ks_train_features = []
    for feature in features:
        pdf1 = data_agreement.loc[data_agreement.signal == 0, feature].values
        pdf2 = data_agreement.loc[data_agreement.signal == 1, feature].values
        ks_train_features.append(ks_2samp_weighted(pdf1, pdf2, weights1=weights_rd_control, weights2=weights))
    return pandas.DataFrame({'Feature': features, 'KS': ks_train_features}).sort('KS')[::-1]

In [ ]:
compute_ks_for_all(train_features, weights_mc_control)

The worst features are `SPDhits`, `FlightDistance`, `IP_p1p2`

In [ ]:
# Divide train on train, test
train_index, test_index = train_test_split(range(len(data)))
train = data.iloc[train_index, :]
test = data.iloc[test_index, :]

In [ ]:
# define function to test model on ks and calculate quality
def test_model(model, features, reweighter=None, reweight_features=None):
    mc_weights = weights_mc_control
    test_weights = numpy.ones(len(test))
    if reweighter is not None:
        mc_weights = reweighter.predict_weights(data_agreement.loc[data_agreement.signal == 1, reweight_features])
        test_weights[test.signal.values == 1] = reweighter.predict_weights(
            test.loc[test.signal == 1, reweight_features])
    probs = model.predict_proba(data_agreement[features])[:, 1]
    pdf1 = probs[data_agreement.signal.values == 0]
    pdf2 = probs[data_agreement.signal.values == 1]
    model_agr = ks_2samp_weighted(pdf1, pdf2, weights1=weights_rd_control, weights2=mc_weights)
    print 'Agreement', model_agr, model_agr < 0.09
    print 'AUC', roc_auc_score(test.signal.values, model.predict_proba(test[features])[:, 1], 
                               sample_weight=test_weights)

### Define GB training with or without reweighter

In [ ]:
def compute_ks_between_mc(model, train_features, reweighter, reweight_features):
    pdf1 = model.predict_proba(test.loc[test.signal == 1, train_features].values)[:, 1]
    pdf2 = model.predict_proba(data_agreement.loc[data_agreement.signal == 1, train_features].values)[:, 1]
    weights1 = numpy.ones(len(pdf1))
    weights2 = numpy.ones(len(pdf2))
    
    if reweighter is not None:
        weights1 = reweighter.predict_weights(test.loc[test.signal == 1, reweight_features].values)
        weights2 = reweighter.predict_weights(data_agreement.loc[data_agreement.signal == 1, reweight_features].values)

    print 'MC vs MC KS:', ks_2samp_weighted(pdf1, pdf2, weights1=weights1, weights2=weights2)

In [ ]:
def train_gb(train_features, reweighter=None, reweight_features=None):
    # define the model GB or another if you want
    gb = ...
    weights = numpy.ones(len(train))
    if reweighter is not None:
        weights[train.signal.values == 1] = reweighter.predict_weights(train.loc[train.signal == 1, reweight_features])
    gb.fit(train[train_features], train['signal'].values, sample_weight=weights)
    test_model(gb, train_features, reweighter, reweight_features)
    compute_ks_between_mc(gb, train_features, reweighter, reweight_features)

### Simple models

#### Simple model on all features

In [ ]:
train_gb(train_features)

#### Remove disagree features

In [ ]:
train_features_agree = list(set(train_features) - {...})

In [ ]:
train_gb(train_features_agree)

### Reweighters for several features

#### define target and original pdfs for reweighters

In [ ]:
target = data_agreement[data_agreement.signal == 0]
original = data_agreement[data_agreement.signal == 1]
reweight_features = [...]

#### Bin Reweighter

In [ ]:
bins_reweighter = reweight.BinsReweighter(...)
bins_reweighter.fit(original[reweight_features], target[reweight_features], target_weight=weights_rd_control)
bins_weights = bins_reweighter.predict_weights(original[reweight_features])
plot_pdf(reweight_features, bins_weights, label='tau BIN ND: ', weights_target=weights_rd_control)

In [ ]:
compute_ks_for_all(train_features, bins_weights)

In [ ]:
train_gb(train_features, bins_reweighter, reweight_features)

#### GB Reweighter

In [ ]:
# do the same as for bin reweighter

In [ ]:
compute_ks_for_all(train_features, ...)

In [ ]:
train_gb(train_features, reweighter, reweight_features)

### Reweight only on `SPDhits` (example of reweighting which doesn't help to pass agreement)

In [ ]:
reweight_features = ['SPDhits']

In [ ]:
reweighter = reweight.GBReweighter(n_estimators=100, learning_rate=0.3, max_depth=5, min_samples_leaf=500, 
                                   gb_args={'subsample': 0.6})
reweighter.fit(original[reweight_features], target[reweight_features], target_weight=weights_rd_control)
gb_weights = reweighter.predict_weights(original[reweight_features])
plot_pdf(reweight_features, gb_weights, label='tau GB ND: ', weights_target=weights_rd_control)

In [ ]:
compute_ks_for_all(train_features, gb_weights)

In [ ]:
train_gb(train_features, reweighter, reweight_features)

# Iterative learning

Try apply iterative learning scheme on $\tau\to3\mu$. Does it help to improve quality?

In [ ]:
variables_1 = ['isolationa', 'isolationb', 'isolationc', 'SPDhits', 'p0_track_Chi2Dof',
              'p1_track_Chi2Dof', 'p2_track_Chi2Dof', 'p0_pt', 'p1_pt', 'p2_pt', 'p0_eta', 'p1_eta', 'p2_eta',
              'p0_IPSig', 'p1_IPSig', 'p2_IPSig']

In [ ]:
variables_2 = list(set(train.columns) - {'mass', 'signal', 'production', 'min_ANNmuon'} - set(variables_1))

## Simple model

In [ ]:
gb = GradientBoostingClassifier(n_estimators=400, max_depth=7, 
                                learning_rate=0.01, min_samples_leaf=50, subsample=0.7, 
                                max_features=8)
gb.fit(train[variables_2], train['signal'].values)
test_model(gb, variables_2)
compute_ks_between_mc(gb, variables_2, None, None)

## Iterative

In [ ]:
...